# Load Model

In [1]:
import config

import cv2
import einops
import numpy as np
import torch
import random

from pytorch_lightning import seed_everything
from annotator.util import resize_image, HWC3
from annotator.canny import CannyDetector
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler


apply_canny = CannyDetector()

config_path = '/home/lolicon/data/dataset/lycoris/model.yaml'
model = create_model(config_path).cpu()


torch_device = 'cuda'

# send to device
model = model.to(torch_device)


/home/lolicon/.virtualenvs/ML/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No module 'xformers'. Proceeding without it.
MultiControlLDM: Running in eps-prediction mode


/home/lolicon/.virtualenvs/ML/lib/python3.10/site-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.19.self_attn.k_proj.weight', 'vision_model.encoder.layers.16.self_attn.k_proj.bias', 'vision_model.encoder.layers.2.mlp.fc1.weight', 'vision_model.encoder.layers.21.layer_norm2.bias', 'vision_model.encoder.layers.17.self_attn.v_proj.bias', 'vision_model.encoder.layers.1.layer_norm1.bias', 'vision_model.encoder.layers.1.mlp.fc1.weight', 'vision_model.encoder.layers.15.self_attn.v_proj.bias', 'vision_model.encoder.layers.21.self_attn.out_proj.bias', 'vision_model.encoder.layers.4.layer_norm1.bias', 'vision_model.encoder.layers.22.self_attn.v_proj.bias', 'vision_model.encoder.layers.23.mlp.fc2.bias', 'vision_model.encoder.layers.22.layer_norm2.bias', 'vision_model.encoder.layers.0.layer_norm2.weight', 'vision_model.encoder.layers.10.self_attn.k_proj.bias', 'vision_model.encoder.layers.0.self_attn.out_proj.bias', 'vision_model.encoder.layers.14.

Loaded model config from [/home/lolicon/data/dataset/lycoris/model.yaml]


# Load Weight

In [2]:

from utils.model_loader import load_state_dict

# weight for unet, encoder, decoder, text embedding 
main_ckpt_path = '../stable-diffusion-webui/models/Stable-diffusion/pastelMixStylizedAnime_pastelMixFull.safetensors'

# weight for control net 
ctrl_pose_path = '../stable-diffusion-webui/models/ControlNet/control_sd15_openpose.pth'

sd_ckpt = load_state_dict(main_ckpt_path)
sd_ctrl = load_state_dict(ctrl_pose_path)
model.load_multi_state_dict(sd_ckpt, pose_model=sd_ctrl)


Loaded state_dict from [../stable-diffusion-webui/models/Stable-diffusion/pastelMixStylizedAnime_pastelMixFull.safetensors]
Loaded state_dict from [../stable-diffusion-webui/models/ControlNet/control_sd15_openpose.pth]


# Create Dataset

In [3]:
from data.dataset import CustomDataset

dataset = CustomDataset('/home/lolicon/data/dataset/lycoris/')


# Training 

### Train

In [5]:
%load_ext autoreload
%autoreload 2

from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar
from torch.utils.data import DataLoader
from cldm.logger import ImageLogger
from data.dataset import MyDataset
from pathlib import Path

import pytorch_lightning as pl

class myLogger(ImageLogger):
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        if not self.disabled:
            self.log_img(pl_module, batch, batch_idx, split="train")

root_dir = '/home/lolicon/data/dataset/lycoris/'
num_epochs = 16
num_gpus = 1
batch_size = 2
logger_freq = 300
learning_rate = 1e-5
sd_locked = True
only_mid_control = False


model.learning_rate = learning_rate
model.sd_locked = sd_locked
model.only_mid_control = only_mid_control


dataloader = DataLoader(dataset, num_workers=0, batch_size=batch_size, shuffle=True)
logger = myLogger(batch_frequency=logger_freq)
trainer = pl.Trainer(gpus=1, precision=16, callbacks=[logger], enable_checkpointing=False)

# ckpt_cb = ModelCheckpoint(dirpath=Path(root_dir) / 'ckpt',
#                           filename='{epoch:d}',
#                           monitor='train/loss',
#                           mode='max',
#                           save_top_k=-1)

# pbar = TQDMProgressBar(refresh_rate=1)
# callbacks = [ckpt_cb, pbar]

# logger = TensorBoardLogger(save_dir="logs",
#                            name='test',
#                            default_hp_metric=False)

# trainer = Trainer(max_epochs=hparams.num_epochs,
#                   callbacks=callbacks,
#                     resume_from_checkpoint=hparams.ckpt_path,
#                     logger=logger,
#                     enable_model_summary=False,
#                     accelerator='auto',
#                     devices=hparams.num_gpus,
#                     num_sanity_val_steps=1,
#                     benchmark=True,
#                     profiler="simple" if hparams.num_gpus==1 else None,
#                     strategy=DDPPlugin(find_unused_parameters=False) if hparams.num_gpus>1 else None)


trainer.fit(model, dataloader)


/home/lolicon/.virtualenvs/ML/lib/python3.10/site-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v2.0.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(
/home/lolicon/.virtualenvs/ML/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/lolicon/.virtualenvs/ML/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not

Epoch 0:   0%|          | 0/20 [00:00<?, ?it/s] 

/home/lolicon/.virtualenvs/ML/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:232: UserWarning: You called `self.log('global_step', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


Data shape for DDIM sampling is (2, 4, 32, 48), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]


: 

: 